# Export workflow to Neurodata Without Borders file and upload to DANDI

In this tutorial, we will walk through exporting two-photon calcium imaging data
stored in a DataJoint pipeline to [Neurodata Without Borders (NWB)](https://www.nwb.org/) file format and
uploading the NWB file to the [DANDI archive](https://dandiarchive.org/).

Please ensure the following before running this notebook for the best experience:
1. Set up an account with the [DANDI archive](https://dandiarchive.org/).
2. Obtain your `DANDI api key` and `Dandiset ID`.

*Before running this notebook, please see the calcium imaging [tutorial](./tutorial.ipynb) notebook to
set up and insert data into the calcium imaging data pipeline.*

In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

import datajoint as dj
from datetime import datetime
import uuid
from workflow.pipeline import *
from element_lab.export import element_lab_to_nwb_dict
from element_animal.export.nwb import subject_to_nwb
from element_session.export.nwb import session_to_nwb
from element_calcium_imaging.export.nwb import imaging_session_to_nwb, write_nwb
from element_interface.dandi import upload_to_dandi

Fetch the `session_key` for the session you want to convert to NWB. 

In [ ]:
session_key = (session.Session & "subject='subject1'").fetch1("KEY")

Define any optional keyword arguments for the NWB file and call the
`imaging_session_to_nwb` function

In [ ]:
nwb_kwargs = {
    "session_start_time": datetime.now(),
    "session_description": "DataJoint Session",
    "identifier": uuid("DJNWBFile"),
}
test_file = imaging_session_to_nwb(session_key=session_key, include_raw_data=False)

Uploading to the DANDI archive requires information in the NWB file that may not be
present when calling `imaging_session_to_nwb` such as the subject's date of birth or
species. We can check for this information within the NWB file and add the information
if it's missing. 

In [ ]:
test_file.subject

In [ ]:
dob, subject_sex = (subject.Subject & session_key).fetch1("subject_birth_date", "sex")
species = "Mus musculus"
test_file.subject.sex = subject_sex
test_file.subject.date_of_birth = dob
test_file.subject.species = species

Save the NWB file to a local directory prior to uploading to DANDI. 

In [ ]:
write_nwb(test_file, "subject1_nwb")

In [ ]:
upload_to_dandi(
    data_directory="./example_data/000683/",
    working_directory="./example_data/",
    dandiset_id=dj.config["custom"]["dandiset_id"],
    staging=False,
    api_key=dj.config["custom"]["dandi.api"],
    sync=False,
)